# AppBuilder-SDK 指定工具调用使用实例

## 一、背景介绍

指定工具调用是为了方便更多用户在AppBuilder官网创建应用之后，希望能直接调用自己定义的工作流或者官方的组件并应用于自己的业务中。具体的调用方法如下面的教程所示。

## 二、实操流程


### 2.1 前置操作简述

- 【必须】登录[百度智能云千帆AppBuilder官网](https://cloud.baidu.com/product/AppBuilder)创建账户。
- 【必须】在[百度智能云千帆AppBuilder控制台](https://console.bce.baidu.com/ai_apaas/dialogHome)左侧菜单栏『我的密钥』页面获取密钥，并复制。
- 【必须】在python3.9及以上的环境中安装`appbuilder-sdk`

### 2.2、创建应用


##### 2.2.1、点击创建应用界面
点击[百度智能云千帆AppBuilder控制台](https://console.bce.baidu.com/ai_apaas/dialogHome)左侧菜单栏『创建应用』，开始我们的系统应用。


#### 2.2.2、填写应用信息，选择工具组件

我们需要首先配置该应用的基本信息，包括 名称、描述、角色指令、开场白、推荐问、能力组件等。

在这里我们首先选择一个组件工具，比如天气查询组件。

<img src="https://bj.bcebos.com/v1/test-tl/toolchoice_create.jpg?authorization=bce-auth-v1%2F6148abe36db84938886e533121ff9628%2F2024-09-23T02%3A47%3A19Z%2F-1%2Fhost%2F3d4b3940d22aa1090a1ffcd9d47e715b526b372b1e9fd4242f1d77ebdde633b6" alt="drawing" width="1000"/>


#### 2.2.3、发布应用

我们可以在页面右上角『发布』按钮发布该应用。

#### 2.2.4 获取已发布应用的ID

在 [百度智能云千帆AppBuilder控制台-我的应用](https://console.bce.baidu.com/ai_apaas/app)页面中，可以查看已发布应用的ID，我们复制该ID，开始后续的代码态操作。

### 2.3 使用SDK调用已发布App

当应用已经发布后，我们可以通过SDK在代码态调用，方便用户集成到自己的系统中，通过自己的系统对外提供服务。

#### 2.3.1 引入AppBuilder-SDK，设置TOKEN，设置APPID

In [1]:
# 引入os模块，引入appbuilder 模块
import os
import appbuilder

# 设置appbuilder的token密钥，从页面上复制粘贴我的密钥，覆盖此处的 "your_appbuilder_token"
os.environ['APPBUILDER_TOKEN'] = "your_appbuilder_token"
# 设置需要调用的app，从页面上复制粘贴应用ID，覆盖此处的 "your_publish_app_id"
app_id = "your_publish_app_id"

print("AppBuilder 模块导入成功！")
print("您的AppBuilder Token为：{}".format(os.environ['APPBUILDER_TOKEN']))
print("您的AppBuilder App ID为：{}".format(app_id))

AppBuilder 模块导入成功！
您的AppBuilder Token为：your_appbuilder_token
您的AppBuilder App ID为：your_publish_app_id


#### 2.3.2 初始化Agent实例，创建会话并对话



In [2]:
# 基于app_id初始化Agent
builder = appbuilder.AppBuilderClient(app_id)
# 创建会话ID
conversation_id = builder.create_conversation()

#### 2.3.3 正常创建会话并对话

- 正常对话下(不采用tool_choice模式)组件的调用受到大模型的控制，参数由大模型生成

In [3]:
#在不采用tool choice的情况下，正常对话APP应用依赖于应用的大模型选择调用组件，而不是直接与组件交互
st = builder.run(conversation_id=conversation_id, query="北京今天的天气", stream=True)

In [4]:
# function_call路由到天气查询组件执行得到组件结果，最终经过总结得到最终结果
#下面结果中天气组件的参数({'city': '北京'})由大模型生成
for k in st.content:
    print(k)
    break

answer='' events=[Event(code=0, message='', status='done', event_type='function_call', content_type='function_call', detail={'text': {'arguments': {'city': '北京'}, 'component_code': 'WeatherQuery', 'component_name': '天气查询'}}, usage=Usage(prompt_tokens=513, completion_tokens=37, total_tokens=550, name='ERNIE Speed-AppBuilder'), tool_calls=None)]


#### 2.3.4 tool_choice方式指定工具参数

In [5]:
#导入指定工具参数类
from appbuilder.core.console.appbuilder_client.data_class import ToolChoiceFunction, ToolChoice
#建立基本的工具调用对象，包括工具名字与参数传递
tool_choice_function = ToolChoiceFunction(name="WeatherQuery", input={"city": "北京天气"})
tool_choice = ToolChoice(type="function", function=tool_choice_function)

#### 2.3.5 tool_choice方式实现对话应用

In [6]:
#请求指定工具tool_choice
st = builder.run(conversation_id=conversation_id, query="北京今天的天气", tool_choice=tool_choice, stream=True)

#### 2.3.6 tool_choice工具调用展示

- 当前模式下由于是直接调用组件，组件的输入参数由输入参数控制

In [7]:
#可以看出，当前天气查询组件的输入参数query({'city': '北京天气'})由tool_choice控制
#此时直接调用天气调用组件而不是大模型function call选择天气组件提供参数
for k in st.content:
    print(k)
    break

answer='' events=[Event(code=0, message='', status='done', event_type='function_call', content_type='function_call', detail={'text': {'arguments': {'city': '北京天气'}, 'component_code': 'WeatherQuery', 'component_name': '天气查询'}}, usage=None, tool_calls=None)]


#### 2.3.7 tool_choice完整结果展示

In [8]:
st = builder.run(conversation_id=conversation_id, query="北京今天的天气", tool_choice=tool_choice, stream=True)
for k in st.content:
    print(k)

answer='' events=[Event(code=0, message='', status='done', event_type='function_call', content_type='function_call', detail={'text': {'arguments': {'city': '北京天气'}, 'component_code': 'WeatherQuery', 'component_name': '天气查询'}}, usage=None, tool_calls=None)]
answer='' events=[Event(code=0, message='', status='preparing', event_type='WeatherQuery', content_type='status', detail={}, usage=None, tool_calls=None)]
answer='' events=[Event(code=0, message='', status='done', event_type='WeatherQuery', content_type='text', detail={'text': '北京  09月23日(星期一) 08:05更新  15.6 ℃  晴 24℃/12℃  东北风1级 | 湿度74%  未来2小时不会下雨 > 北京今明天云量逐渐增多 明夜有小雨 > 堪比台风降雨!冷空气“撞”上季风 华南暴雨大暴雨持续 局地特大暴雨 推荐 图集 昨天 9/22 23/11℃  今天 9/23 24/12℃  星期二 9/24 24/15℃  48小时预报 09:00 11:00 13:00 15:00 17:00 19:00 21:00 23:00 01:00 03:00 05:00 07:00 09:00 11:00 13:00 15:00 17:00 19:00 21:00 23:00 01:00 03:00 05:00 07:00 09:00 17℃20℃22℃23℃21℃20℃17℃16℃14℃13℃12℃13℃16℃21℃23℃23℃23℃21℃19℃18℃16℃15℃15℃16℃19℃ <3级 <3级 <3级 <3级 <3级 <3级 <3级 <3级 <3级 <3级 <3级 <3级 <3级